<a href="https://colab.research.google.com/github/hargurjeet/Adhoc-Activities/blob/main/Fake_review_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt # plotting
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as py
from wordcloud import WordCloud

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
import zipfile

def unzip_file(zip_file_path, output_dir):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(output_dir)

# Example usage:
zip_file_path = '/content/ReviewerNamewise.zip'  # Replace this with the path to your zip file
output_dir = '/content/'  # Replace this with the directory where you want to extract the files
unzip_file(zip_file_path, output_dir)

In [5]:
import os
import pandas as pd

def read_and_append_csv_files(folder_path):
    all_data = pd.DataFrame()  # Initialize an empty DataFrame to store the combined data

    # List all files in the folder
    file_list = os.listdir(folder_path)

    # Loop through each file
    for file_name in file_list:
        if file_name.endswith('.csv'):  # Check if the file is a CSV file
            file_path = os.path.join(folder_path, file_name)  # Get the full file path
            data = pd.read_csv(file_path)  # Read the CSV file
            all_data = all_data.append(data, ignore_index=True)  # Append the data to the DataFrame

    return all_data

# Example usage:
folder_path = r'/content/ReviewerNamewise/NormalCase'  # Replace this with the path to your folder containing CSV files
combined_data = read_and_append_csv_files(folder_path)
doubtfulcase = read_and_append_csv_files(r'/content/ReviewerNamewise/Doubtfulcase')

<ipython-input-5-179b5b707250>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data, ignore_index=True)  # Append the data to the DataFrame
<ipython-input-5-179b5b707250>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data, ignore_index=True)  # Append the data to the DataFrame
<ipython-input-5-179b5b707250>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data, ignore_index=True)  # Append the data to the DataFrame
<ipython-input-5-179b5b707250>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data, ignore_index=True)  # A

In [6]:
doubtfulcase.shape

(113, 5)

In [7]:
combined_data.shape

(644, 5)

In [8]:
from joblib import load

# Load the model from the file
loaded_model = load('/content/drive/MyDrive/Documents/models.joblib')

In [9]:
loaded_model

[XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=2, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               n_estimators=100, n_jobs=None, num_parallel_tree=None,
               predictor=None, random_state=None, ...),
 RandomForestClassifier()]

In [10]:
import string
# Define a function to preprocess the review text
def preprocess_text(text):
    # Convert the text to lowercase and split it into words
    words = text.lower().split()

    # Remove punctuation and numbers from the words
    words = [word.translate(str.maketrans('', '', string.punctuation + string.digits)) for word in words]

    # Remove stop words from the words
    stopwords = set(nltk.corpus.stopwords.words('english'))
    words = [word for word in words if word not in stopwords]

    # Stem the words using a Porter stemmer
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]

    # Return the preprocessed text as a list of words
    return ' '.join(words)

combined_data['Text'] = combined_data['Text'].apply(preprocess_text)
doubtfulcase['Text'] = doubtfulcase['Text'].apply(preprocess_text)

In [11]:
combined_data['Text']

0                                                   good
1              pathet home deliveri servic commit fulfil
2                                                 execel
3      go place monthli bulk shop locat soni world si...
4                                                  excel
                             ...                        
639    fresh product budget friendli clean neat place...
640    good true  excel setup arrang food groceri dai...
641                           one best store malleshwram
642                                           nice store
643    use favorit megastor us varieti offer gener fr...
Name: Text, Length: 644, dtype: object

In [14]:
import gensim
# Train the Word2Vec model on the preprocessed review text
sentences = [review.split() for review in combined_data['Text']]
model = gensim.models.Word2Vec(sentences, vector_size=100, min_count=1, workers=4)

In [15]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

def get_word2vec_embedding(text):
    # Tokenize the text into words
    words = word_tokenize(text)

    # Train a word2vec model on the tokenized words
    model = Word2Vec([words], min_count=1, vector_size=100)

    # Get the word2vec embeddings for the words
    word_vectors = model.wv

    # Return the word2vec embeddings
    return word_vectors

In [19]:
doc_embeddings = []
for sentence in combined_data['Text']:
    words = sentence.split()
    vectors = [model.wv.get_vector(word) for word in words if word in model.wv.key_to_index]
    if vectors:
        mean_vector = np.mean(vectors, axis=0)
        doc_embeddings.append(mean_vector)
    else:
        doc_embeddings.append(np.zeros(model.vector_size))
combined_data['doc_embeddings']=(doc_embeddings)

In [17]:
len(doc_embeddings)

644

In [20]:
combined_data.head()

,Store Name,Reviewer_Name,Review_date,Text,primary_no,doc_embeddings
0,Village Hyper Market Marathahalli,JAMSHEER,2012-05-10,good,jd9972,"[-0.008255966, 0.05298534, 0.010494903, 0.0059..."
1,AMEX Supermarket,Satnam Negi,2015-12-12,pathet home deliveri servic commit fulfil,jd5,"[-0.005382773, 0.030652396, 0.002788984, -0.00..."
2,DMart,Raju chinny,2019-06-27,execel,jd1833,"[0.00039608718, -0.005508865, -0.0022076727, 0..."
3,DMart,AJIT S,2023-02-13,go place monthli bulk shop locat soni world si...,jd4336,"[-0.005949817, 0.030320056, 0.0006092803, -0.0..."
4,De Needs Hyper Bazzar,appu,2013-09-09,excel,jd648,"[-0.0065989294, 0.009457003, 0.004214515, -0.0..."


In [22]:
# new_comment="This is an example sentence."
# pre_new_comment = preprocess_text(new_comment)
# embeddings = get_word2vec_embedding(pre_new_comment)

# # Data type of array element
# Data_type = float

# # This cause Value error
# np_array = np.array(embeddings, dtype=Data_type)


# embeddings = np.reshape(embeddings, (1, -1))
# Predict the class label for the new record
# predicted_class = classifiers[-1].predict(embeddings)
combined_data['predicted_class'] = loaded_model[-1].predict(doc_embeddings)
# Print the predicted class

In [23]:
combined_data

,Store Name,Reviewer_Name,Review_date,Text,primary_no,doc_embeddings,predicted_class
0,Village Hyper Market Marathahalli,JAMSHEER,2012-05-10,good,jd9972,"[-0.008255966, 0.05298534, 0.010494903, 0.0059...",0
1,AMEX Supermarket,Satnam Negi,2015-12-12,pathet home deliveri servic commit fulfil,jd5,"[-0.005382773, 0.030652396, 0.002788984, -0.00...",0
2,DMart,Raju chinny,2019-06-27,execel,jd1833,"[0.00039608718, -0.005508865, -0.0022076727, 0...",0
3,DMart,AJIT S,2023-02-13,go place monthli bulk shop locat soni world si...,jd4336,"[-0.005949817, 0.030320056, 0.0006092803, -0.0...",0
4,De Needs Hyper Bazzar,appu,2013-09-09,excel,jd648,"[-0.0065989294, 0.009457003, 0.004214515, -0.0...",0
...,...,...,...,...,...,...,...
639,DMart,SRUTHY,2023-10-02,fresh product budget friendli clean neat place...,jd750,"[-0.004339671, 0.021075498, 0.0033622934, -0.0...",0
640,DMart,NextGen Digitalindia,2017-10-02,good true excel setup arrang food groceri dai...,jd1161,"[-0.0063088643, 0.035283513, 0.0041019935, -0....",0
641,Spar Hypermarket,channa,2022-02-08,one best store malleshwram,jd8154,"[-0.007436881, 0.039499003, 0.0021828243, 0.00...",0
642,DMart,Ahmed,2016-12-18,nice store,jd3173,"[-0.011912028, 0.044320498, 0.0032190126, -0.0...",0
